In [ ]:
import matplotlib,aplpy,numpy,glob
from astropy.wcs import WCS
from astropy.io import fits
from astropy.table import Table
from astropy.table import QTable
import matplotlib.pyplot as plt
import common_functions as cf
from astropy import units
from astropy.coordinates import SkyCoord
import aplpy

In [ ]:
font = {'size'   : 14, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1

In [ ]:
contour_scheme = numpy.array([1.,2.,4.,8.,16.,32.,64.])

In [ ]:
GBT_point = QTable([[92, 48, 58, 88, 91, 37, 68, 79, 93, 7, 16, 25, 31, 97, 23, 30, 100, 15, 26, 10, 33, 90, 40],
                    ['22h35m57.500s','10h37m45.600s','11h42m10.700s','20h52m24.000s','22h09m10.400s','09h13m35.600s','13h53m26.000s','15h59m11.800s','23h15m24.200s',
                     '00h39m23.900s','02h09m31.300s','03h20m43.700s','05h01m38.300s','23h47m22.900s','03h07m06.400s','04h36m28.600s','00h01m20.700s','02h07m38.900s',
                     '03h21m54.200s','01h26m13.000s','05h10m47.900s','22h02m05.600s','09h38m54.500s'],
                    ['+33d57m35.00s','-27d04m49.00s','+10d18m21.00s','-05d45m00.00s','-27d47m45.00s','+30d00m51.00s','+40d19m08.00s','+20d45m30.00s','+18d58m59.00s',
                     '+00d52m41.00s','-10d09m30.00s','-01d03m06.00s','-04d15m25.00s','-02d19m34.00s','-09d35m07.00s','-02d49m56.00s','+13d07m57.00s','+02d08m17.00s',
                     '-13d38m45.00s','+34d42m41.00s','+18d02m05.00s','-31d58m00.00s','-04d51m07.00s']],
                   names = ['HCG', 'ra', 'dec'])
GBT_point.add_index('HCG')

In [ ]:
HCGs = Table.read('tables/HCGs.vo', format='votable')
HCGs.add_index('HCG')

HCG_mems = Table.read('tables/HCG_members.vo', format='votable')

HCGs['logMHI_gals'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_exfs'] = numpy.zeros(len(HCGs))+numpy.nan

HCG_mems['logMHI'] = numpy.zeros(len(HCG_mems))+numpy.nan

In [ ]:
def split_overlay(HCG, ra_cen, dec_cen, width, dist, opt_survey='DECaLS'):
    
    fits_file_list = glob.glob(f'SoFiA_masks/HCG{HCG}/separated_features/*.fits')
    src_cat = 'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG)
    sofia_mom0 = 'SoFiA_masks/HCG{0}/HCG{0}_mom0.fits'.format(HCG)
    pbcube = 'SoFiA_masks/HCG{0}/HCG{0}_HI.pbcor.fits'.format(HCG)
    if 'POSS' in opt_survey:
        optical_image = opt_survey+'_images/HCG{0}_R.fits'.format(HCG)
    else:
        optical_image = opt_survey+'_images/r_fits/HCG{0}_r.fits'.format(HCG)
        
    members = list(HCG_mems['name'][HCG_mems['HCG'] == HCG])
    src_cat = Table.read(src_cat)
    rms = numpy.mean(src_cat['rms'])

    for pb_cubelet in fits_file_list:

        cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(pb_cubelet,True,True)
        
        n_HI = (rms*abs(cube_dv)*4.*numpy.log(2.)*2.356E-7)/(numpy.pi*bmaj*bmin*(numpy.pi/(180.*3600.))**2.)
        N_HI = n_HI*cf.Msol/(cf.mp*(cf.pc*10.)**2.)

        #Catch the error that SlicerAstro sometimes writes an incorrect value in pixel (0,0,0)
        if abs(cube[0,0,0]) > 1.:
            cube[0,0,0] = 0.

        mom0 = numpy.nansum(cube,axis=0)*abs(cube_dv)

        if 'mem_mom0' not in locals():
            mem_mom0 = numpy.zeros(numpy.shape(mom0))
            non_mom0 = numpy.zeros(numpy.shape(mom0))
            tid_mom0 = numpy.zeros(numpy.shape(mom0))
        
        if numpy.shape(mom0) != numpy.shape(mem_mom0):
            print('ERROR: Cubelets not all the same dimensions.')
            return None
            
        #Identify what object is classified as
        group_feature = False
        feat_code = ['br','ex','tl','cl']
        for feature in feat_code:
            if feature in pb_cubelet:
                inx = pb_cubelet.index(feature)
                print(f'HCG{HCG}'+pb_cubelet[inx:inx+len(feature)+1]+' is a non-galactic feature.')
                group_feature = True
                break

        group_member = False
        if not group_feature:
            for mem in members:
                if mem+'.fits' in pb_cubelet:
                    print(f'{mem} is a member of HCG{HCG}.')
                    group_member = True
                    break

        #Add exceptions for HCGs 47, 49, 57, and 100
        if ("abcd" in pb_cubelet) or ("acd" in pb_cubelet) or ("ab" in pb_cubelet):
            group_member = True
            inx1 = pb_cubelet.index('features/')+9
            inx2 = pb_cubelet.index('.fits')
            print(pb_cubelet[inx1:inx2]+f' is part of HCG{HCG}.')
            
        if group_member:
            mem_mom0 += mom0
        elif group_feature:
            tid_mom0 += mom0
        else:
            non_mom0 += mom0
    
    hdu = fits.open(fits_file_list[0])
    wcs = WCS(hdu[0].header,naxis=2)
    

    cen_pos = [ra_cen,dec_cen]
    fig_wid = width
    
    min_contour = 4.*rms*abs(cube_dv)
    print(min_contour)
    
    #Initialise figure
    f = aplpy.FITSFigure(optical_image, figsize=(8.,8.), dimensions=[0, 1])

    #Re-centre and set size (identical to above command)
    f.recenter(cen_pos[0], cen_pos[1], radius=fig_wid/2.)

    #Overlay contours
    hdu = fits.PrimaryHDU(mem_mom0)
    hdu.header.update(wcs.to_header())
    f.show_contour(hdu,colors=['blue'],
                         levels=[-1.*min_contour],linestyle='--')
    f.show_contour(hdu,colors=['blue'],
                         levels=min_contour*contour_scheme)
    hdu = fits.PrimaryHDU(tid_mom0)
    hdu.header.update(wcs.to_header())
    f.show_contour(hdu,colors=['green'],
                         levels=[-1.*min_contour],linestyle='--')
    f.show_contour(hdu,colors=['green'],
                         levels=min_contour*contour_scheme)
    hdu = fits.PrimaryHDU(non_mom0)
    hdu.header.update(wcs.to_header())
    f.show_contour(hdu,colors=['red'],
                         levels=[-1.*min_contour],linestyle='--')
    f.show_contour(hdu,colors=['red'],
                         levels=min_contour*contour_scheme)
    #Contour values in Jy km/s per beam

    #Display r-band image
    if 'POSS' not in opt_survey:
        f.show_colorscale(cmap='Greys',vmin=1E-4,vmax=2.5,stretch='log')
    else:
        f.show_colorscale(cmap='Greys',pmin=0.1,pmax=99.8,stretch='log')

    #Add beam
    hdu = fits.open(pbcube)
    f.show_ellipses(xw=cen_pos[0]+0.45*fig_wid, yw=cen_pos[1]-0.45*fig_wid, width=hdu[0].header['BMIN'], height=hdu[0].header['BMAJ'], angle=hdu[0].header['BPA'], facecolor='g', lw=0)

    #Add scalebar
    f.add_scalebar((180./numpy.pi)*0.02/dist, corner='bottom right', lw=4, label="20 kpc")
    
    #Title
    f.set_title(f'HCG {HCG}')
    
    if HCG in GBT_point['HCG']:
        GBT_pointing = SkyCoord(GBT_point.loc[HCG]['ra'], GBT_point.loc[HCG]['dec'], frame='icrs', equinox='J2000')
        f.show_circles(xw=GBT_pointing.ra.deg,yw=GBT_pointing.dec.deg,radius=9.1/120.,edgecolor='orange', lw=3, ls='dotted')
        
    f.savefig('overlays/HCG{}_split_overlay.pdf'.format(HCG),dpi=100)
    f.savefig('overlays/HCG{}_split_overlay.png'.format(HCG),dpi=100)

In [ ]:
HCG = 2
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 7
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 10
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'], 'SDSS')

In [ ]:
HCG = 15
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 16
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.45, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 19
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.4, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 22
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 23
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 25
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 26
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 30
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 31
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 33
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.1, HCGs.loc[HCG]['dist'], 'POSS')

In [ ]:
HCG = 37
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 40
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 47
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 48
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'], 'POSS')

In [ ]:
HCG = 49
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 54
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 56
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 57
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 58
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 59
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 61
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.275, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 68
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.3, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 71
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 79
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.1, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 88
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 90
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.35, HCGs.loc[HCG]['dist'], 'POSS')

In [ ]:
HCG = 91
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'], 'POSS')

In [ ]:
HCG = 92
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.125, HCGs.loc[HCG]['dist'], 'SDSS')

In [ ]:
HCG = 93
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 96
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 97
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 100
split_overlay(HCG, HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.25, HCGs.loc[HCG]['dist'])